# Hits-EQ get the counts of each sequence step by step

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import itertools

## Set parameters
- Reference sequence: the sequence exist in our target sequence but not in the randomized region.
- sequence_start: Where your sequence start. Remember in python. In python, the first location is 0 and here 2 means the 3rd nucleotide.

In [13]:

test_seq = "GAGCTTGAATCTCAC"  # sequence which used to filter what we want
sequence_start = 2
nTs = ["A","C","G","T"]
# Where your sequence start. Remember in python,
# the first location is 0 and here 2 means the 3rd nucleotide.
number = 2
where_dimer = 0
howmany = 6
len_test = len(test_seq)
dimer="NO"

In [11]:
def quality_check(sequence, reference):
    seq_len = len(sequence)
    ref_len = len(reference)
    n = 0
    for i in range(ref_len):
        if sequence[i] != reference[i]:
            n += 1
    return n

def preprocess_sequence(input_seq):
        try:
            processed_sequence = input_seq[sequence_start : (sequence_start + len_test)]
            # Start to check the sequence, if mismatch <=2, then print the sequence into the file
            mismatch = quality_check(processed_sequence, test_seq)
            if mismatch <= 2:
                 return input_seq
        except:
            return "False"

def generate_nts(nT, seq_length):
    products = itertools.product(nT,repeat=seq_length)
    return pd.Series(["".join(x) for x in products])

In [12]:
#create the two and six nucleotide
majorN = generate_nts(nTs, howmany)
twoN = generate_nts(nTs, 2)
dict_major = {x:0 for x in majorN}
dict_dimer ={x:0 for x in twoN}

def read_count(input_seq):
        try:
            dict_major[input_seq[number:number+howmany]] += 1
        except:
            pass
def read_count_diNT(input_seq):
        try:
            dict_dimer[input_seq[where_dimer:where_dimer+2]] += 1
        except:
            pass

## Start Processing

In [7]:
with open("filename", "r") as f:
    while True:
        header = f.readline().strip()
        seq = f.readline().strip()
        plus = f.readline().strip()
        quality = f.readline().strip()
        if not header:
            break
        try:
            processed_sequence = preprocess_sequence(seq)

        except:
            pass

        if processed_sequence is not "False":
                read_count(processed_sequence)
                if dimer is "YES":
                     read_count_diNT(processed_sequence)
        else:
             pass

In [6]:
result_major = pd.DataFrame(dict_major.items(), columns=["sequence", "counts"])
result_diNT =  pd.DataFrame(dict_dimer.items(), columns=["dimer", "dimer_counts"])
export_table = pd.concat(result_major, result_diNT, axis=1)
export_table.to_csv("result.csv")